### Maximization of the Log-Likelihood of State Space Models on the Limit Order Book

In [2]:
import pandas as pd
import numpy as np

In [2]:
import os
os.add_dll_directory('C:\Aorda\PSG\lib')
import psgpython as psg 